In [1]:
import os
import utils
import random
import dataset
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.contrib.eager as tfe

In [2]:
# configs
tfe.enable_eager_execution()
%matplotlib inline

In [10]:
# load data and display random example
X=tf.convert_to_tensor(datasets.load_dataset("ShapeNet"))
input_shape=X.shape[1:]
X.shape

[load_dataset] loading ShapeNet
[construct_paths] for dir ShapeNet
[fetch_renders_from_disk] in dir ./Renders
[fetch_renders_from_disk] returning 10 renders from ./Renders


TensorShape([Dimension(10), Dimension(512), Dimension(512), Dimension(3)])

In [4]:
# net parameters
conv_filter_count = [96, 128, 256, 256, 256, 256]
deconv_filter_count = [128, 128, 128, 64, 32, 2]